In [44]:
import pandas as pd
import numpy as np
import os
from itertools import groupby
from operator import itemgetter

In [2]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure

In [3]:
output_notebook()

Loading BokehJS ...

In [4]:
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, RangeTool
from bokeh.plotting import figure, show

In [42]:
output_path = "/home/pdutta/Data/Rekha_data/Enhancer/WholeGenome/chr21/predicted_enhancer"

In [5]:
prob_arr = "/home/pdutta/Data/Rekha_data/Enhancer/WholeGenome/chr21/prob.npy"

In [6]:
with open(prob_arr, 'rb') as f:
    a = np.load(f)

In [7]:
a

array([0., 0., 0., ..., 0., 0., 0.])

In [8]:
position_arr = np.arange(5010001, 5010001+len(a))

In [9]:
position_arr

array([ 5010001,  5010002,  5010003, ..., 46699975, 46699976, 46699977])

In [10]:
len(a)

41689977

In [11]:
df = pd.DataFrame({'position':position_arr, 'probability':a})

In [12]:
df

,position,probability
0,5010001,0.0
1,5010002,0.0
2,5010003,0.0
3,5010004,0.0
4,5010005,0.0
...,...,...
41689972,46699973,0.0
41689973,46699974,0.0
41689974,46699975,0.0
41689975,46699976,0.0


In [13]:
df[df['probability']>0.4]

,position,probability
41283088,46293089,0.400331
41283089,46293090,0.403272
41283090,46293091,0.407003
41283091,46293092,0.404322
41283092,46293093,0.404322
41283093,46293094,0.401752
41283094,46293095,0.401752
41283095,46293096,0.401752
41283105,46293106,0.401483
41283106,46293107,0.404599


In [14]:
df.iloc[0:10]['position']

0    5010001
1    5010002
2    5010003
3    5010004
4    5010005
5    5010006
6    5010007
7    5010008
8    5010009
9    5010010
Name: position, dtype: int64

In [15]:
start = 41280088
end = 41283113
positions = np.array(df['position'])
source = ColumnDataSource(data=dict(position=df.iloc[start:end]['position'], probability=df.iloc[start:end]['probability']))
p = figure(height=300, width=1400,
           x_axis_type="linear", x_axis_location="above",
           background_fill_color="#efefef", x_range=(positions[start], positions[end]))
p.xgrid.grid_line_width=1
p.ygrid.grid_line_width=1
p.xaxis.axis_label = 'position'
p.yaxis.axis_label = 'Probaibilty value'
p.line('position', 'probability', source=source)
#show(p)



select = figure(title="Drag the middle and edges of the selection box to change the range above",
                height=200, width=1000, y_range=p.y_range,
                x_axis_type="linear", y_axis_type=None,
                toolbar_location=None, background_fill_color="#efefef")

range_tool = RangeTool(x_range=p.x_range)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2

select.line('position', 'probability', source=source)
select.ygrid.grid_line_color = None
select.add_tools(range_tool)
select.toolbar.active_multi = range_tool
show(column(p, select))

In [52]:
np.arange(0.4, 0.1, -0.05)

array([0.4 , 0.35, 0.3 , 0.25, 0.2 , 0.15, 0.1 ])

In [54]:
threshold_list = np.arange(0.4, 0.1, -0.01)
for threshold in threshold_list:
    df_out = pd.DataFrame(columns= ['start_pos','end_pos','len','prob_values','avg_prob'])
    index=0
    df_temp = df[df['probability']>=threshold]
    #print(df_temp)
    windows = df_temp['position'].to_list()
    bins = []
    for k, g in groupby(enumerate(windows),lambda ix : ix[0] - ix[1]):
        temp_list = list(map(itemgetter(1), g))
        #print(temp_list)
        probab_values = df_temp[(df_temp['position'] >= temp_list[0]) & (df_temp['position'] <= temp_list[-1])]['probability'].to_list()
        mean = df_temp[(df_temp['position'] >= temp_list[0]) & (df_temp['position'] <= temp_list[-1])]['probability'].mean()
        #print(mean)
        df_out.loc[index, ['start_pos']]=  temp_list[0]
        df_out.loc[index, ['end_pos']]=  temp_list[-1]
        df_out.loc[index, ['len']]=  len(temp_list)
        df_out.loc[index, ['prob_values']]=  str(probab_values)
        df_out.loc[index, ['avg_prob']]=  mean
        index +=1
    print(threshold , len(df_out))
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    df_out.to_csv("{}/predicted_enhancer_thre={}.csv".format(output_path, threshold), index=False)

0.4 2
0.39 3
0.38 6
0.37 10
0.36 10
0.35 7
0.33999999999999997 9
0.32999999999999996 8
0.31999999999999995 4
0.30999999999999994 8
0.29999999999999993 12
0.2899999999999999 14
0.2799999999999999 13
0.2699999999999999 14
0.2599999999999999 22
0.2499999999999999 20
0.23999999999999988 29
0.22999999999999987 26
0.21999999999999986 44
0.20999999999999985 47
0.19999999999999984 72
0.18999999999999984 109
0.17999999999999983 169
0.16999999999999982 249
0.1599999999999998 346
0.1499999999999998 493
0.1399999999999998 653
0.12999999999999978 914
0.11999999999999977 1309
0.10999999999999976 1785
0.09999999999999976 2511


In [22]:
df_temp['position'][46293089:46293092]('probab')

Series([], Name: position, dtype: int64)

In [30]:
df_temp[(df_temp['position'] >= 46293089) & (df_temp['position'] <= 46293096)]

,position,probability
41283088,46293089,0.400331
41283089,46293090,0.403272
41283090,46293091,0.407003
41283091,46293092,0.404322
41283092,46293093,0.404322
41283093,46293094,0.401752
41283094,46293095,0.401752
41283095,46293096,0.401752


AttributeError: 'list' object has no attribute 'mean'